In [1]:
import tensorflow as tf
import collections
import numpy as np

In [2]:
num_steps = 30
batch_size = 10
lstm_size = 5
embed_size = 30
vocabSz = 10000

In [3]:
# Read data
path = '/Users/cytology/simple-examples/data/ptb.train.txt'
with tf.gfile.GFile(path) as f:
    data = f.read().replace("/n", "<eos>").split()
batch_len = (len(data) - 1) // batch_size
epoch_size = batch_len // num_steps
# Convert words to IDs
counter = sorted(collections.Counter(data).items(), 
                 key=lambda x: -x[1])[:vocabSz] # sort by frequency
wwords, _ = list(zip(*counter))
word_to_ID = dict(zip(wwords, range(len(wwords))))
data = [word_to_ID[word] for word in data if word in word_to_ID]
raw_data = tf.convert_to_tensor(data, tf.int32)
batch_data = tf.reshape(raw_data[:batch_size * batch_len], 
                        [batch_size, batch_len])

In [4]:
# Placeholder for the inputs in a given iteration.
words = tf.placeholder(tf.int32, [batch_size, num_steps])
targets = tf.placeholder(tf.int32, [batch_size, num_steps])

lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
# Initial state of the LSTM memory.
initial_state = state = lstm.zero_state(batch_size, dtype=tf.float32)

for i in range(num_steps):
    # The value of state is updated after processing each batch of words.
    output, state = lstm(words[:, i], state)

    # The LSTM output can be used to make next word predictions
    logits = tf.matmul(output, softmax_w) + softmax_b
    xEnt = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=targets[:,i])
    loss = tf.reduce_mean(xEnt)

final_state = state


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


ValueError: Input 0 of layer basic_lstm_cell_1 is incompatible with the layer: expected ndim=2, found ndim=1. Full shape received: [10]

In [ ]:
# A numpy array holding the state of LSTM after each batch of words.
numpy_state = initial_state.eval()
total_loss = 0.0
for batch in epoch:
    start = batch * num_steps
    end = start + num_steps
    current_batch_of_words = batch_data[:,start:end]
    anss = batch_data[:, start+1:end+1]
    numpy_state, current_loss = session.run([final_state, loss],
        # Initialize the LSTM state from the previous iteration.
        feed_dict={initial_state: numpy_state, words: current_batch_of_words, targets:anss})
    total_loss += current_loss